In [3]:
import duckdb
import sys
import pandas as pd
import os
from datetime import datetime

In [19]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [24]:
arquivo = 'z0019_1.csv'
data_ingestao = datetime.now()

df = pd.read_csv(f'../landing/{arquivo}', sep=';', encoding='latin-1')
df['nome_arquivo'] = arquivo
df['data_ingestao'] = data_ingestao
df.head()

,NATBR,MAKTX,WERKS,MAINS,LASBT,nome_arquivo,data_ingestao
0,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-13 16:48:29.181069
1,1002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-13 16:48:29.181069
2,1003,PREGO,BT10,100,150,z0019_1.csv,2026-02-13 16:48:29.181069


In [25]:
con.execute('''
CREATE TABLE IF NOT EXISTS bronze_produtos (
    NATBR VARCHAR,
    MAKTX VARCHAR,
    WERKS VARCHAR,
    MAINS VARCHAR,
    LABST VARCHAR,
    nome_arquivo VARCHAR,
    data_ingestao TIMESTAMP
)
''')

In [26]:
con.execute("INSERT INTO bronze_produtos SELECT * FROM df")

In [30]:
resultado = con.execute("SELECT * FROM bronze_produtos").fetchdf()
resultado.head(6)

CatalogException: Catalog Error: Table with name bronze_produtos does not exist!
Did you mean "bronze_z0019"?

LINE 1: SELECT * FROM bronze_produtos
                      ^

In [29]:
con.execute("drop table if exists bronze_z0019")
con.execute("ALTER TABLE bronze_produtos RENAME TO bronze_z0019")

In [28]:
con.execute("ALTER TABLE bronze_produtos RENAME TO bronze_z0019")

CatalogException: Catalog Error: Could not rename "bronze_produtos" to "bronze_z0019": another entry with this name already exists!

In [33]:
resultado = con.execute("SELECT * FROM bronze_z0019").fetchdf()
resultado.head(6)

,NATBR,MAKTX,WERKS,MAINS,LABST,nome_arquivo,data_ingestao
0,1004,SERRA,BT50,100,100,z0019_2.csv,2026-02-13 16:48:01.184980
1,1005,MACHADO,BT50,100,1500,z0019_2.csv,2026-02-13 16:48:01.184980
2,1003,PREGO,BT10,100,150,z0019_2.csv,2026-02-13 16:48:01.184980
3,1001,PARAFUSO,BT10,100,100,z0019_1.csv,2026-02-13 16:48:29.181069
4,1002,MARTELO,BT50,100,1500,z0019_1.csv,2026-02-13 16:48:29.181069
5,1003,PREGO,BT10,100,150,z0019_1.csv,2026-02-13 16:48:29.181069


In [34]:
con.close()